In [34]:
import requests
import json
import pprint
import pandas as pd

url = "https://dahiti.dgfi.tum.de/api/v2/download-water-level/"

args = {
    'api_key': '0068D5A859F71E204F7C0A7D5BEA1D2ADB495E22EECB95C51F871337A4B79453',
    'dahiti_id': 40216,  # Masinga Dam ID
    'format': 'json'     # json format response
}

response = requests.get(url, params=args)

if response.status_code == 200:
    if args['format'] == "ascii":
        data_ascii = response.text
        print(data_ascii)
    elif args['format'] == "json":
        data_json = json.loads(response.text)
        pprint.pprint(data_json)
        
        # ---- Save JSON data to CSV ----
        # Extract the relevant part: usually the 'data' key contains rows
        records = data_json.get('data', [])
        
        # Convert list of dicts to pandas DataFrame
        df = pd.DataFrame(records)
        
        # Save DataFrame to CSV file
        df.to_csv('masinga_water_levels.csv', index=False)
        print("Saved water level data to masinga_water_levels.csv")
        
    elif args['format'] == "netcdf":
        path_netcdf = "/tmp/" + str(args['dahiti_id']) + "_water_level_altimetry.nc"
        print('writing ... ' + path_netcdf)
        with open(path_netcdf, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print('done!')
    elif args['format'] == "csv":
        data_csv = response.text
        print(data_csv)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


{'continent': 'Africa',
 'country': 'Kenya',
 'creation_date': '2025-08-12 18:18:18',
 'dahiti_id': '40216',
 'data': [{'datetime': '2018-12-28 07:29:44', 'wse': 1052.33, 'wse_u': 0.003},
          {'datetime': '2019-01-24 07:29:45', 'wse': 1051.334, 'wse_u': 0.001},
          {'datetime': '2019-02-20 07:29:46', 'wse': 1049.362, 'wse_u': 0.009},
          {'datetime': '2019-03-19 07:29:50', 'wse': 1045.828, 'wse_u': 0.059},
          {'datetime': '2019-04-15 07:29:51', 'wse': 1043.745, 'wse_u': 0.066},
          {'datetime': '2019-05-12 07:29:50', 'wse': 1044.557, 'wse_u': 0.002},
          {'datetime': '2019-06-08 07:29:44', 'wse': 1046.0, 'wse_u': 0.01},
          {'datetime': '2019-07-05 07:29:45', 'wse': 1045.812, 'wse_u': 0.002},
          {'datetime': '2019-08-01 07:29:47', 'wse': 1043.824, 'wse_u': 0.008},
          {'datetime': '2019-08-28 07:29:46', 'wse': 1041.137, 'wse_u': 0.003},
          {'datetime': '2019-10-21 07:29:42', 'wse': 1043.68, 'wse_u': 0.046},
          {'date

In [39]:
import pandas as pd

# Step 1: Load the data (assuming CSV file)
df = pd.read_csv('masinga_water_levels.csv')

# Step 2: Convert datetime column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Step 3: Check for missing values
print("Missing values:\n", df.isnull().sum())

# Step 4: Remove duplicates if any
df = df.drop_duplicates()

# Step 5: Sort by datetime
df = df.sort_values(by='datetime').reset_index(drop=True)

# Step 6: Convert wse and wse_u to float (if not already)
df['wse'] = pd.to_numeric(df['wse'], errors='coerce')
df['wse_u'] = pd.to_numeric(df['wse_u'], errors='coerce')

# Step 7: Final check for any NaNs after conversions
print("Missing values after conversion:\n", df.isnull().sum())

# Optional: filter data between 2018-01-01 and 2020-12-31 for your project scope
start_date = '2018-01-01'
end_date = '2020-12-31'
df_filtered = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]

print(df_filtered.head())


Missing values:
 datetime    0
wse         0
wse_u       0
dtype: int64
Missing values after conversion:
 datetime    0
wse         0
wse_u       0
dtype: int64
             datetime       wse  wse_u
0 2018-12-28 07:29:44  1052.330  0.003
1 2019-01-24 07:29:45  1051.334  0.001
2 2019-02-20 07:29:46  1049.362  0.009
3 2019-03-19 07:29:50  1045.828  0.059
4 2019-04-15 07:29:51  1043.745  0.066


In [3]:
import requests
import pandas as pd

# NASA POWER API parameters
latitude = -0.87   # Example: Masinga Dam latitude
longitude = 37.53  # Example: Masinga Dam longitude
start = 2018
end = 2020

url = f"https://power.larc.nasa.gov/api/temporal/monthly/point"
params = {
    "parameters": "PRECTOT",  # Total monthly precipitation
    "community": "AG",
    "longitude": longitude,
    "latitude": latitude,
    "start": start,
    "end": end,
    "format": "JSON"
}

# Make request
response = requests.get(url, params=params)

# Print raw JSON to debug
print(response.json())


{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [37.53, -0.87, 1227.42]}, 'properties': {'parameter': {'PRECTOTCORR': {'201801': 0.16, '201802': 0.41, '201803': 7.23, '201804': 11.2, '201805': 3.72, '201806': 1.39, '201807': 0.71, '201808': 0.73, '201809': 0.47, '201810': 1.46, '201811': 2.34, '201812': 4.44, '201813': 2.86, '201901': 0.84, '201902': 0.21, '201903': 0.47, '201904': 1.72, '201905': 1.69, '201906': 2.12, '201907': 0.72, '201908': 0.64, '201909': 0.67, '201910': 5.45, '201911': 5.1, '201912': 5.08, '201913': 2.07, '202001': 3.02, '202002': 1.57, '202003': 2.93, '202004': 8.79, '202005': 1.58, '202006': 2.18, '202007': 0.6, '202008': 0.8, '202009': 1.94, '202010': 3.05, '202011': 5.21, '202012': 1.72, '202013': 2.77}}}, 'header': {'title': 'NASA/POWER Source Native Resolution Monthly and Annual', 'api': {'version': 'v2.7.3', 'name': 'POWER Monthly and Annual API'}, 'sources': ['MERRA2'], 'fill_value': -999.0, 'time_standard': 'LST', 'start': '20180101', 'e

In [38]:
print(df.columns)


Index(['datetime', 'wse', 'wse_u'], dtype='object')


In [40]:
import requests
import pandas as pd

lat, lon = -0.75, 37.5

def fetch_weather_data(start_date, end_date):
    url = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?"
        f"parameters=T2M,PRECTOTCORR&community=AG&longitude={lon}&latitude={lat}"
        f"&start={start_date}&end={end_date}&format=JSON"
    )
    response = requests.get(url)
    data = response.json()
    
    temp_data = data['properties']['parameter']['T2M']
    precip_data = data['properties']['parameter']['PRECTOTCORR']
    
    dates = []
    temps = []
    precip = []
    
    for date_str in temp_data.keys():
        dates.append(pd.to_datetime(date_str, format='%Y%m%d'))
        temps.append(temp_data[date_str])
        precip.append(precip_data.get(date_str, None))
        
    df = pd.DataFrame({
        'date': dates,
        'temperature_C': temps,
        'precipitation_mm': precip
    })
    return df

# Fetch 2018 data
df_2018 = fetch_weather_data('20180101', '20181231')
# Fetch 2019 data
df_2019 = fetch_weather_data('20190101', '20191231')
# Fetch 2020 data
df_2020 = fetch_weather_data('20200101', '20201231')

# Combine all years
weather_df = pd.concat([df_2018, df_2019, df_2020], ignore_index=True)

print(weather_df.shape)
print(weather_df.tail())


(1096, 3)
           date  temperature_C  precipitation_mm
1091 2020-12-27          19.63              0.02
1092 2020-12-28          19.54              0.20
1093 2020-12-29          20.68              0.28
1094 2020-12-30          20.66              5.18
1095 2020-12-31          18.60             21.98


In [41]:
import pandas as pd

# Step 1: Load data
weather_df = pd.read_csv('masinga_weather_2018_2020.csv')

# Step 2: Check columns
print(weather_df.columns)

# Step 3: Convert date to datetime
weather_df['date'] = pd.to_datetime(weather_df['date'])

# Step 4: Check for missing values
print(weather_df.isnull().sum())

# Step 5: Basic stats
print(weather_df.describe())


Index(['date', 'temperature_C', 'precipitation_mm'], dtype='object')
date                0
temperature_C       0
precipitation_mm    0
dtype: int64
                      date  temperature_C  precipitation_mm
count                 1096    1096.000000       1096.000000
mean   2019-07-02 12:00:00      20.198677          3.706770
min    2018-01-01 00:00:00      15.770000          0.000000
25%    2018-10-01 18:00:00      19.267500          0.480000
50%    2019-07-02 12:00:00      20.160000          1.450000
75%    2020-04-01 06:00:00      21.072500          3.850000
max    2020-12-31 00:00:00      24.970000         60.810000
std                    NaN       1.562804          6.579551


In [19]:
weather_df.to_csv('masinga_weather_2018_2020.csv', index=False)


In [42]:
import pandas as pd
import requests

# Step 1: Download the CSV file
csv_url = "https://open.africa/dataset/9b94fe50-9d75-4b92-be00-6354c6e6cc88/resource/111a3d9d-c676-4d5a-9600-d5acb1250b87/download/population-houseshold-data.csv"
response = requests.get(csv_url)
with open('kenya_population_data.csv', 'wb') as f:
    f.write(response.content)

# Step 2: Load CSV into a DataFrame
df = pd.read_csv('kenya_population_data.csv')

# Step 3: Filter population data for Masinga Sub-County
masinga_population = df[df['Sub-County'] == 'Masinga']

# Print population info for 2019
pop_2019 = masinga_population['Population'].values[0]
print(f"Masinga Sub-County population in 2019 (census): {pop_2019}")

# Step 4: Estimate population for 2018 and 2020 (using 2.3% growth rate)
growth_rate = 0.023
pop_2018 = pop_2019 / (1 + growth_rate)
pop_2020 = pop_2019 * (1 + growth_rate)

print(f"Estimated population in 2018: {int(pop_2018)}")
print(f"Estimated population in 2020: {int(pop_2020)}")


KeyError: 'Sub-County'

In [45]:
import pandas as pd

# Read CSV, skipping metadata rows
population_df = pd.read_csv('kenya_population_data.csv', skiprows=6)

# Check first few rows and columns (optional)
print(population_df.head())
print(population_df.columns)

# Make sure Population, Households, Avg Household size are numeric
population_df['Population'] = pd.to_numeric(population_df['Population'], errors='coerce')
population_df['No.of Households'] = pd.to_numeric(population_df['No.of Households'], errors='coerce')
population_df['Avg Household size'] = pd.to_numeric(population_df['Avg Household size'], errors='coerce')

# Remove any rows with NaN in 'name'
population_df = population_df.dropna(subset=['name'])

# Filter for Machakos county only
population_machakos = population_df[population_df['name'].str.strip().str.lower() == 'machakos'].reset_index(drop=True)

# Check shape and preview
print(population_machakos.shape)
print(population_machakos.head())


         name  Population  No.of Households  Avg Household size
0       Kenya    47213282          12143913                  39
1     Mombasa     1190987            378422                  31
2       Kwale      858748            173176                  50
3      Kilifi     1440958            298472                  48
4  Tana River      314710             68242                  46
Index(['name', 'Population', 'No.of Households', 'Avg Household size'], dtype='object')
(1, 4)
       name  Population  No.of Households  Avg Household size
0  Machakos     1414022            402466                  35


In [47]:
import pandas as pd

# Load weather data
weather_df = pd.read_csv('masinga_weather_2018_2020.csv')
weather_df['date'] = pd.to_datetime(weather_df['date'])

# Load population data with skiprows=6 because of metadata in the first 6 rows
population_df = pd.read_csv('kenya_population_data.csv', skiprows=6)
population_df['Population'] = pd.to_numeric(population_df['Population'], errors='coerce')
population_df['name'] = population_df['name'].str.strip()

# Filter for Machakos county only
population_machakos = population_df[population_df['name'] == 'Machakos'].reset_index(drop=True)

# Load water level data
water_df = pd.read_csv('masinga_water_levels.csv')

# Fix the column name if necessary (your earlier error was about '.datetime' column missing)
# Check columns to find the correct datetime column name:
print(water_df.columns)

# Assuming column name is 'datetime' or similar, if not adjust accordingly
if '.datetime' in water_df.columns:
    datetime_col = '.datetime'
elif 'datetime' in water_df.columns:
    datetime_col = 'datetime'
else:
    # Use first column as datetime, if appropriate
    datetime_col = water_df.columns[0]

# Convert datetime column to datetime type
water_df[datetime_col] = pd.to_datetime(water_df[datetime_col])

# Create a date-only column for merging
water_df['date'] = water_df[datetime_col].dt.date
weather_df['date_only'] = weather_df['date'].dt.date

# Merge weather and water levels on date
merged_df = pd.merge(weather_df, water_df, left_on='date_only', right_on='date', how='inner')

# Drop duplicate date columns to clean up
merged_df = merged_df.drop(columns=['date_only', 'date_y'])

# Add Machakos population as a constant column for analysis
merged_df['population_machakos'] = population_machakos.loc[0, 'Population']

# Inspect the final merged dataframe
print(merged_df.head())
print(merged_df.shape)


Index(['datetime', 'wse', 'wse_u'], dtype='object')
      date_x  temperature_C  precipitation_mm            datetime       wse  \
0 2018-12-28          20.34              0.69 2018-12-28 07:29:44  1052.330   
1 2019-01-24          21.40              0.00 2019-01-24 07:29:45  1051.334   
2 2019-02-20          22.73              0.27 2019-02-20 07:29:46  1049.362   
3 2019-03-19          23.91              0.04 2019-03-19 07:29:50  1045.828   
4 2019-04-15          23.77              0.06 2019-04-15 07:29:51  1043.745   

   wse_u  population_machakos  
0  0.003              1414022  
1  0.001              1414022  
2  0.009              1414022  
3  0.059              1414022  
4  0.066              1414022  
(25, 7)


In [48]:
# Rename date_x to just date for clarity
merged_df = merged_df.rename(columns={'date_x': 'date'})

# If you want to keep datetime from water data but align it with date only, you can keep both or drop one
# For example, drop 'datetime' if you prefer just the date:
merged_df = merged_df.drop(columns=['datetime'])


In [50]:
merged_df.head()


,date,temperature_C,precipitation_mm,wse,wse_u,population_machakos
0,2018-12-28,20.34,0.69,1052.330,0.003,1414022
1,2019-01-24,21.40,0.00,1051.334,0.001,1414022
2,2019-02-20,22.73,0.27,1049.362,0.009,1414022
3,2019-03-19,23.91,0.04,1045.828,0.059,1414022
4,2019-04-15,23.77,0.06,1043.745,0.066,1414022
